In [2]:
import plotly.graph_objects as go
import geopandas as gpd
import json

# Загрузка данных
with open('mean_temp_period1.geojson', 'r') as f:
    data_period1 = json.load(f)

with open('mean_temp_period2.geojson', 'r') as f:
    data_period2 = json.load(f)

with open('evolution_temp.geojson', 'r') as f:
    data_evolution = json.load(f)

# Загрузка границ Европы
europe_geojson_path = 'europe.geojson'  # Путь к GeoJSON с границами Европы
gdf_europe = gpd.read_file(europe_geojson_path)

# Удаление недействительных геометрий
gdf_europe = gdf_europe[gdf_europe.geometry.notnull()]
gdf_europe = gdf_europe[gdf_europe.geometry.type.isin(['Polygon', 'MultiPolygon'])]

# Преобразование данных в GeoDataFrame и фильтрация точек внутри границ Европы
def extract_and_filter_data(geojson, europe_boundary):
    points = gpd.GeoDataFrame(
        {
            "value": [feature['properties']['value'] for feature in geojson['features']],
            "geometry": gpd.points_from_xy(
                [feature['geometry']['coordinates'][0] for feature in geojson['features']],
                [feature['geometry']['coordinates'][1] for feature in geojson['features']]
            )
        }
    )
    filtered_points = gpd.clip(points, europe_boundary)
    return (
        filtered_points.geometry.x.tolist(),
        filtered_points.geometry.y.tolist(),
        filtered_points['value'].tolist()
    )

# Преобразование границ Европы в единое геометрическое множество
europe_boundary = gdf_europe.unary_union

# Извлечение и фильтрация данных
lon1, lat1, values1 = extract_and_filter_data(data_period1, europe_boundary)
lon2, lat2, values2 = extract_and_filter_data(data_period2, europe_boundary)
lon_evo, lat_evo, values_evo = extract_and_filter_data(data_evolution, europe_boundary)

# Добавление названий стран из поля NAME_FREN
gdf_europe["hover_text"] = gdf_europe["NAME_FREN"]

# Список всех столиц Европы с координатами и переводами на французский (оставляем без изменений)
european_capitals = [
    {"name": "Paris (France)", "lon": 2.3522, "lat": 48.8566},
    {"name": "Berlin (Allemagne)", "lon": 13.405, "lat": 52.52},
    {"name": "Madrid (Espagne)", "lon": -3.7038, "lat": 40.4168},
    {"name": "Rome (Italie)", "lon": 12.4964, "lat": 41.9028},
    {"name": "Oslo (Norvège)", "lon": 10.7522, "lat": 59.9139},
    {"name": "Berne (Suisse)", "lon": 7.4474, "lat": 46.9481},
    {"name": "Londres (Royaume-Uni)", "lon": -0.1278, "lat": 51.5074},
    {"name": "Stockholm (Suède)", "lon": 18.0686, "lat": 59.3293},
    {"name": "Helsinki (Finlande)", "lon": 24.9384, "lat": 60.1695},
    {"name": "Kiev (Ukraine)", "lon": 30.5238, "lat": 50.4501},
    {"name": "Vilnius (Lituanie)", "lon": 25.2797, "lat": 54.6872},
    {"name": "Riga (Lettonie)", "lon": 24.1052, "lat": 56.9496},
    {"name": "Varsovie (Pologne)", "lon": 21.0122, "lat": 52.2297},
    {"name": "Minsk (Biélorussie)", "lon": 27.5615, "lat": 53.9006},
    {"name": "Belgrade (Serbie)", "lon": 20.4573, "lat": 44.8176},
    {"name": "Istanbul (Turquie)", "lon": 28.9784, "lat": 41.0082},
    {"name": "Chisinau (Moldavie)", "lon": 28.8575, "lat": 47.0105},
    {"name": "Athènes (Grèce)", "lon": 23.7275, "lat": 37.9838},
    {"name": "Prague (République tchèque)", "lon": 14.4378, "lat": 50.0755},
    {"name": "Budapest (Hongrie)", "lon": 19.0402, "lat": 47.4979},
    {"name": "Bucarest (Roumanie)", "lon": 26.1025, "lat": 44.4268},
    {"name": "Bruxelles (Belgique)", "lon": 4.3517, "lat": 50.8503},
    {"name": "Amsterdam (Pays-Bas)", "lon": 4.9041, "lat": 52.3676},
    {"name": "Dublin (Irlande)", "lon": -6.2603, "lat": 53.3498},
    {"name": "Vienne (Autriche)", "lon": 16.3738, "lat": 48.2082},
    {"name": "Tirana (Albanie)", "lon": 19.8189, "lat": 41.3275},
    {"name": "Skopje (Macédoine du Nord)", "lon": 21.43, "lat": 41.9981},
    {"name": "Sofia (Bulgarie)", "lon": 23.3219, "lat": 42.6977},
    {"name": "Ljubljana (Slovénie)", "lon": 14.5058, "lat": 46.0569},
    {"name": "Sarajevo (Bosnie-Herzégovine)", "lon": 18.4131, "lat": 43.8563},
    {"name": "Tallinn (Estonie)", "lon": 24.7536, "lat": 59.437},
    {"name": "Bratislava (Slovaquie)", "lon": 17.1077, "lat": 48.1486},
    {"name": "Podgorica (Monténégro)", "lon": 19.2594, "lat": 42.441},
    {"name": "Lisbonne (Portugal)", "lon": -9.139, "lat": 38.7223},
    {"name": "Copenhague (Danemark)", "lon": 12.5683, "lat": 55.6761},
    {"name": "Moscou (Russie)", "lon": 37.6173, "lat": 55.7558},
    {"name": "Zagreb (Croatie)", "lon": 15.9819, "lat": 45.815}
]

# Создание базовой карты
fig = go.Figure()

# Добавление слоя для периода 1950-1986 (как точки)
fig.add_trace(go.Scattermapbox(
    lon=lon1,
    lat=lat1,
    mode='markers',
    marker=dict(
        size=8,
        color=values1,
        colorscale=[[0, 'blue'], [0.18, 'white'], [0.19, 'white'], [1, 'red']],
        colorbar=dict(
            title=dict(
                text="Température (°C)",
                font=dict(family="Montserrat", size=12, color="black")
            ),
            tickvals=[-5, 0, 5, 10, 15, 20],
            ticktext=["-5", "0", "5", "10", "15", "20"]
        ),
        showscale=True
    ),
    hovertext=[f"{value:.1f}" for value in values1],
    hoverinfo="text",
    name="1950-1986",
    visible=True,
    showlegend=False
))

# Добавление слоя для периода 1987-2023 (как точки)
fig.add_trace(go.Scattermapbox(
    lon=lon2,
    lat=lat2,
    mode='markers',
    marker=dict(
        size=8,
        color=values2,
        colorscale=[[0, 'blue'], [0.12, 'white'], [0.13, 'white'], [1, 'red']],
        colorbar=dict(
            title=dict(
                text="Température (°C)",
                font=dict(family="Montserrat", size=12, color="black")
            ),
            tickvals=[-5, 0, 5, 10, 15, 20],
            ticktext=["-5", "0", "5", "10", "15", "20"]
        ),
        showscale=True
    ),
    hovertext=[f"{value:.1f}" for value in values2],
    hoverinfo="text",
    name="1987-2023",
    visible=False,
    showlegend=False
))

# Добавление слоя для изменения температуры (как точки)
fig.add_trace(go.Scattermapbox(
    lon=lon_evo,
    lat=lat_evo,
    mode='markers',
    marker=dict(
        size=8,
        color=values_evo,
        colorscale=[[0, 'blue'], [0.19, 'white'], [0.2, 'white'], [1, 'red']],
        colorbar=dict(
            title=dict(
                text="Évolution Température (°C)",
                font=dict(family="Montserrat", size=12, color="black")
            )
        ),
        showscale=True
    ),
    hovertext=[f"{value:.1f}" for value in values_evo],
    hoverinfo="text",
    name="Évolution",
    visible=False,
    showlegend=False
))

# Добавление столиц Европы (оставляем без изменений)
fig.add_trace(go.Scattermapbox(
    lon=[capital['lon'] for capital in european_capitals],
    lat=[capital['lat'] for capital in european_capitals],
    mode='markers',
    marker=dict(size=7, color='black'),
    hovertext=[capital['name'] for capital in european_capitals],
    hoverinfo="text",
    showlegend=False
))

# Добавление границ Европы напрямую в фигуру
for _, row in gdf_europe.iterrows():
    if row.geometry.geom_type == "Polygon":
        x, y = row.geometry.exterior.xy
        fig.add_trace(go.Scattermapbox(
            lon=list(x),
            lat=list(y),
            mode="lines",
            line=dict(width=1, color="grey"),
            hoverinfo="none",
            showlegend=False
        ))
    elif row.geometry.geom_type == "MultiPolygon":
        for poly in row.geometry.geoms:
            x, y = poly.exterior.xy
            fig.add_trace(go.Scattermapbox(
                lon=list(x),
                lat=list(y),
                mode="lines",
                line=dict(width=1, color="grey"),
                hoverinfo="none",
                showlegend=False
            ))

# Общее количество трасс (слоев)
total_traces = len(fig.data)

# Настройка кнопок для переключения слоев
fig.update_layout(
    mapbox=dict(
        style="carto-positron",
        center={"lat": 50, "lon": 10},
        zoom=3
    ),
    title={
        "text": "Températures Moyennes 1950-1986",
        "x": 0.5,
        "font": {"family": "Montserrat"}
    },
    margin=dict(l=250, r=250, t=50, b=50),
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            x=0.28,
            y=0.08,
            buttons=[
                dict(
                    label="1950-1986",
                    method="update",
                    args=[
                        {"visible": [True, False, False] + [True]*(total_traces - 3)},
                        {"title.text": "Températures Moyennes en Europe (1950-1986)"}
                    ]
                ),
                dict(
                    label="1987-2023",
                    method="update",
                    args=[
                        {"visible": [False, True, False] + [True]*(total_traces - 3)},
                        {"title.text": "Températures Moyennes en Europe (1987-2023)"}
                    ]
                ),
                dict(
                    label="Évolution",
                    method="update",
                    args=[
                        {"visible": [False, False, True] + [True]*(total_traces - 3)},
                        {"title.text": "Évolution des Températures Moyennes (1987-2023 vs 1950-1986)"}
                    ]
                )
            ]
        )
    ]
)

# Сохранение карты
fig.write_html("evolution_map.html")
